In [1]:
from matplotlib.backends.backend_pdf import PdfPages
import h5py                                    # To open snapshots
import numpy as np                             # Numerical library
# For quickly finding all snapshot paths
from glob import glob
from pyread_eagle import EagleSnapshot         # Eagle read routine
# Useful to get age of the universe and thus integration times
from astropy.cosmology import FlatLambdaCDM
# from eagle_database.database import Database   # Eagle database code

# plotting
import matplotlib.pyplot as plt
from matplotlib import gridspec
%matplotlib inline


plt.style.use("./shao.mplstyle")
fontSize = 15
lineWidth = 1.5

colors = [u'#1f77b4', u'#ff7f0e', u'#2ca02c', u'#d62728', u'#9467bd', u'#8c564b', u'#e377c2', u'#7f7f7f',
          u'#bcbd22', u'#17becf']


In [4]:
R_max = 300.  # physical kpc
Mstar_min = 1.  # in solar masses

5

In [5]:
# specify for which entries below to calculate the satellite positions at each redshift
#systems = [16, 17, 18, 19]
systems = [1]

# central galaxy index (i.e.) position in the Galform file (one entry per system)
# only needed for the systems where the central is not a type=0 galaxy with the largest halo mass
# index of the galaxy in the Galform file dataset 'Output001'
centralIndeces = [
    0,    # M1_zcut7
    0,    # M2_zcut7
    0,    # M3_zcut7
    518,  # M4_zcut7
    79,   # M5_zcut7
    0,    # M1_7DM_GAS
    387,  # M2_7DM_GAS
    0,    # M3_7DM_GAS
    581,  # M4_7DM_GAS
    0,    # M5_7DM_GAS
    0,    # M1_MR_DMO
    0,    # M2_MR_DMO
    0,    # M3_MR_DMO
    597,  # M4_MR_DMO
    109,  # M5_MR_DMO
    0,    # M1_HR_DMO
    0,    # M2_HR_DMO
    0,    # M3_HR_DMO
    596,  # M4_HR_DMO
    0     # M5_HR_DMO
]

# path to the input files
root_galaxyFiles = "/home/zhaox/Magpie/Galform/"
list_galaxyFiles = [
    "M1_zcut7/galaxies.hdf5",
    "M2_zcut7/galaxies.hdf5",
    "M3_zcut7/galaxies.hdf5",
    "M4_zcut7/galaxies.hdf5",
    "M5_zcut7/galaxies.hdf5",
    "M1_7DM_GAS/galaxies.hdf5",
    "M2_7DM_GAS/galaxies.hdf5",
    "M3_7DM_GAS/galaxies.hdf5",
    "M4_7DM_GAS/galaxies.hdf5",
    "M5_7DM_GAS/galaxies.hdf5",
    "M1_MR_DMO/galaxies.hdf5",
    "M2_MR_DMO/galaxies.hdf5",
    "M3_MR_DMO/galaxies.hdf5",
    "M4_MR_DMO/galaxies.hdf5",
    "M5_MR_DMO/galaxies.hdf5",
    "M1_HR_DMO/galaxies.hdf5",
    "M2_HR_DMO/galaxies.hdf5",
    "M3_HR_DMO/galaxies.hdf5",
    "M4_HR_DMO/galaxies.hdf5",
    "M5_HR_DMO/galaxies.hdf5"
]

# path to the input particle data files
root_subhaloFiles = "/home/zhaox/Magpie/"
list_subhaloFiles = [
    "M1_zcut7/groups_%03i_z???p???/",
    "M2_zcut7/groups_%03i_z???p???/",
    "M3_zcut7/groups_%03i_z???p???/",
    "M4_zcut7/groups_%03i_z???p???/",
    "M5_zcut7/groups_%03i_z???p???/",
    "M1_7DM_GAS/groups_%03i_z???p???/",
    "M2_7DM_GAS/groups_%03i_z???p???/",
    "M3_7DM_GAS/groups_%03i_z???p???/",
    "M4_7DM_GAS/groups_%03i_z???p???/",
    "M5_7DM_GAS/groups_%03i_z???p???/",
    "M1_MR_DMO/groups_%03i_z???p???/",
    "M2_MR_DMO/groups_%03i_z???p???/",
    "M3_MR_DMO/groups_%03i_z???p???/",
    "M4_MR_DMO/groups_%03i_z???p???/",
    "M5_MR_DMO/groups_%03i_z???p???/",
    "M1_HR_DMO/groups_%03i_z???p???/",
    "M2_HR_DMO/groups_%03i_z???p???/",
    "M3_HR_DMO/groups_%03i_z???p???/",
    "M4_HR_DMO/groups_%03i_z???p???/",
    "M5_HR_DMO/groups_%03i_z???p???/"
]

# path to the output files
root_outputFiles = "./satellite_orbits_Galform/"
list_outputFiles = [
    "M1_MR.hdf5",
    "M2_MR.hdf5",
    "M3_MR.hdf5",
    "M4_MR.hdf5",
    "M5_MR.hdf5",
    "M1_7DM_GAS.hdf5",
    "M2_7DM_GAS.hdf5",
    "M3_7DM_GAS.hdf5",
    "M4_7DM_GAS.hdf5",
    "M5_7DM_GAS.hdf5",
    "M1_MR_DMO.hdf5",
    "M2_MR_DMO.hdf5",
    "M3_MR_DMO.hdf5",
    "M4_MR_DMO.hdf5",
    "M5_MR_DMO.hdf5",
    "M1_HR_DMO.hdf5",
    "M2_HR_DMO.hdf5",
    "M3_HR_DMO.hdf5",
    "M4_HR_DMO.hdf5",
    "M5_HR_DMO.hdf5"
]

# path to the output files for figures
plotFigures = True
root_plotFiles = root_outputFiles
list_plotFiles = [
    "M1_MR_orbits.pdf",
    "M2_MR_orbits.pdf",
    "M3_MR_orbits.pdf",
    "M4_MR_orbits.pdf",
    "M5_MR_orbits.pdf",
    "M1_7DM_GAS_orbits.pdf",
    "M2_7DM_GAS_orbits.pdf",
    "M3_7DM_GAS_orbits.pdf",
    "M4_7DM_GAS_orbits.pdf",
    "M5_7DM_GAS_orbits.pdf",
    "M1_MR_DMO_orbits.pdf",
    "M2_MR_DMO_orbits.pdf",
    "M3_MR_DMO_orbits.pdf",
    "M4_MR_DMO_orbits.pdf",
    "M5_MR_DMO_orbits.pdf",
    "M1_HR_DMO_orbits.pdf",
    "M2_HR_DMO_orbits.pdf",
    "M3_HR_DMO_orbits.pdf",
    "M4_HR_DMO_orbits.pdf",
    "M5_HR_DMO_orbits.pdf"
]

if systems is None:
    systems = list(range(len(centralIndeces)))


array([2, 3])

In [ ]:
# function for reading in the Subfind data
def readSubhalomass(subhaloFile, gal_subID):
    """Reads the group data from a Subfind file."""
    subfile_basename = 'eagle_subfind_tab_???_z???p???.%i.hdf5'
    # create array for subhalo mass
    numObjs = gal_subID.shape[0]
    numSnaps = gal_subID.shape[1]
    gal_Msub = np.zeros((numObjs, numSnaps), np.float64)
    for i in range(0, numObjs):
        for j in range(0, numSnaps):
            if j == 0 and i % 10 == 0:
                print("\t\t %ith galaxy finished" % (i))
            if gal_subID[i, j] < 0:
                continue
            ID_temp = gal_subID[i, j]
            a = divmod(ID_temp, 10**15)  # a[0] > 0 if no subhalo exists
            b = divmod(a[1], 10**12)    # b[0] is snapnum
            # c[0] is the file number and c[1] is the index of the subhalo
            c = divmod(b[1], 10**8)
            snapname = subhaloFile % b[0]
            subfilename = snapname + subfile_basename
            inputFiles = glob(subfilename % c[0])
            # find and read the file
            with h5py.File(inputFiles[0], 'r') as hf:
                #           print( hf['Header'].attrs.keys() )    # returns the names of all the attributes in the header
                HubbleParam = hf['Header'].attrs['HubbleParam']
                numGroups = hf['Header'].attrs['TotNgroups']
                numSubgroups = hf['Header'].attrs['TotNsubgroups']
                gal_Msub[i, j] = np.array(
                    hf['Subhalo/Mass'])[c[1]] * 1.e10 / HubbleParam
    return gal_Msub, numGroups, numSubgroups

In [1]:
divmod(199,20)

(9, 19)